In [1]:
# 피마 인디언 당뇨병 데이터세트를 이용하여 다음에 답하세요.

# 1. 각 Feature의 최소값이 0, 최대값이 1이 되도록 MinMaxScaler를 사용하여
#    변환하세요.(10)

# 2. SVC, KNN, Logistic Regression을 소프트 보팅 방식으로 앙상블 학습을 하되,   SVC의 C 파라메터와 LR의 C 파라메터를 최적화해서 분류를 시도하세요.(20)

# 3. 결정트리를 기반으로 하는 앙상블 학습기 3종류를 이용하여, 정확도 / 정밀도 /  
#    재현율을 구하세요.(20)

> 1. 각 Feature의 최소값이 0, 최대값이 1이 되도록 MinMaxScaler를 사용하여
 변환하세요.(10)

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('../01.Intro/pima-indians-diabetes.csv' ,skiprows=9, header=None) 
df.columns = ['P', 'G', 'BP', 'S', 'I', 'BMI', 'D', 'Age', 'Target']
df.head()

,P,G,BP,S,I,BMI,D,Age,Target
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.isna().sum().sum()

0

In [5]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [6]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(
    X, y,stratify=y,
    test_size=0.2,
    random_state=2023
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((614, 8), (154, 8), (614,), (154,))

In [7]:
from sklearn.preprocessing import MinMaxScaler 
X_mm = MinMaxScaler().fit_transform(X)

>2. SVC, KNN, Logistic Regression을 소프트 보팅 방식으로 앙상블 학습을 하되,   SVC의 C 파라메터와 LR의 C 파라메터를 최적화해서 분류를 시도하세요.(20)

In [8]:
from sklearn.preprocessing import StandardScaler 
X_std = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_std, y, stratify=y, test_size=0.2, random_state=2023
)

In [9]:
from sklearn.svm import SVC 
svc = SVC(probability=True,random_state=2023)

In [10]:
from sklearn.linear_model import LogisticRegression 
lrc = LogisticRegression(random_state=2023)

In [11]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

In [15]:
svc.fit(X_train, y_train)
svc.predict_proba(X_test)

lrc.fit(X_train, y_train)
lrc.predict_proba(X_test)

knn.fit(X_train, y_train)
knn.predict_proba(X_test)

array([[0.8, 0.2],
       [0.4, 0.6],
       [0.4, 0.6],
       [0.4, 0.6],
       [0.6, 0.4],
       [1. , 0. ],
       [0.8, 0.2],
       [0.4, 0.6],
       [0. , 1. ],
       [1. , 0. ],
       [1. , 0. ],
       [0.6, 0.4],
       [0.4, 0.6],
       [0.8, 0.2],
       [0.2, 0.8],
       [0.8, 0.2],
       [0.4, 0.6],
       [0.6, 0.4],
       [0.6, 0.4],
       [0.6, 0.4],
       [1. , 0. ],
       [0.6, 0.4],
       [0.4, 0.6],
       [0.4, 0.6],
       [1. , 0. ],
       [0.4, 0.6],
       [0.8, 0.2],
       [0. , 1. ],
       [0.2, 0.8],
       [0.6, 0.4],
       [1. , 0. ],
       [0.8, 0.2],
       [0.6, 0.4],
       [1. , 0. ],
       [0.8, 0.2],
       [0.2, 0.8],
       [0.6, 0.4],
       [0.6, 0.4],
       [0.8, 0.2],
       [0.2, 0.8],
       [1. , 0. ],
       [0.2, 0.8],
       [0.6, 0.4],
       [0.2, 0.8],
       [1. , 0. ],
       [0.2, 0.8],
       [1. , 0. ],
       [0.4, 0.6],
       [0.4, 0.6],
       [0.8, 0.2],
       [0.2, 0.8],
       [0.2, 0.8],
       [0.2,

In [16]:
params = {
    'LRC__C': [0.1, 1, 10],
    'SVC__C': [0.1, 1, 10]
}

In [17]:
from sklearn.ensemble import VotingClassifier
voc = VotingClassifier(
    estimators=[('LRC', lrc), ('SVC', svc), ('KNN', knn)],
    voting='soft'
)

In [18]:
from sklearn.model_selection import GridSearchCV 
grid_voc = GridSearchCV(voc, params, scoring='accuracy', cv=5)
grid_voc.fit(X_train, y_train)
grid_voc.best_params_

{'LRC__C': 0.1, 'SVC__C': 1}

In [19]:
grid_voc.best_estimator_.score(X_test, y_test)

0.7792207792207793

> 3. 결정트리를 기반으로 하는 앙상블 학습기 3종류를 이용하여, 정확도 / 정밀도 /  
  재현율을 구하세요.(20)

In [30]:
# 정확도(accuracy) 비교 
from sklearn.metrics import accuracy_score
# accuracy_score(y_test)
from sklearn.metrics import precision_score, recall_score
# 정밀도, 재현율
# precision_score(y_test)
# 재현율(recall)
# recall_score(y_test)

In [32]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=156)
dtc.fit(X_train, y_train)
dtc_pred = dtc.predict(X_test)
acc_dtc = accuracy_score(y_test, dtc_pred)
prec_dtc = precision_score(y_test, dtc_pred)
reca_dtc = recall_score(y_test, dtc_pred)

In [33]:
from sklearn.ensemble import RandomForestClassifier 
rfc = RandomForestClassifier(random_state=2023)
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)
acc_rfc = accuracy_score(y_test, rfc_pred)
prec_rfc = precision_score(y_test, rfc_pred)
reca_rfc = recall_score(y_test, rfc_pred)

In [36]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
pred_xgb = xgb.predict(X_test)
acc_xgb = accuracy_score(y_test, pred_xgb)
prec_xgb = precision_score(y_test, pred_xgb)
reca_xgb = recall_score(y_test, pred_xgb)

In [39]:
from lightgbm import LGBMClassifier 
lgb = LGBMClassifier()
evals = [(X_test, y_test)]
lgb.fit(X_train, y_train, eval_set=evals, eval_metric='logloss', verbose=True)
pred_lgb = lgb.predict(X_test)
acc_lgb = accuracy_score(y_test, pred_lgb)
prec_lgb = precision_score(y_test, pred_lgb)
reca_lgb = recall_score(y_test, pred_lgb)

[1]	valid_0's binary_logloss: 0.615506
[2]	valid_0's binary_logloss: 0.5971
[3]	valid_0's binary_logloss: 0.578737
[4]	valid_0's binary_logloss: 0.563357
[5]	valid_0's binary_logloss: 0.547289
[6]	valid_0's binary_logloss: 0.536826
[7]	valid_0's binary_logloss: 0.526478
[8]	valid_0's binary_logloss: 0.5196
[9]	valid_0's binary_logloss: 0.509382
[10]	valid_0's binary_logloss: 0.502299
[11]	valid_0's binary_logloss: 0.499422
[12]	valid_0's binary_logloss: 0.49336
[13]	valid_0's binary_logloss: 0.48997
[14]	valid_0's binary_logloss: 0.489997
[15]	valid_0's binary_logloss: 0.489147
[16]	valid_0's binary_logloss: 0.488191
[17]	valid_0's binary_logloss: 0.488111
[18]	valid_0's binary_logloss: 0.488663
[19]	valid_0's binary_logloss: 0.486831
[20]	valid_0's binary_logloss: 0.485895
[21]	valid_0's binary_logloss: 0.486498
[22]	valid_0's binary_logloss: 0.486552
[23]	valid_0's binary_logloss: 0.489444
[24]	valid_0's binary_logloss: 0.489001
[25]	valid_0's binary_logloss: 0.487257
[26]	valid_0's 

c:\Users\YONSAI\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


In [42]:
# print(f'dtc -> 정확도:{acc_dtc:.4f}, 정밀도:{prec_dtc:.4f}, 재현율:{reca_dtc:.4f}')
print(f'rfc -> 정확도:{acc_rfc:.4f}, 정밀도:{prec_rfc:.4f}, 재현율:{reca_rfc:.4f}')
print(f'xgb -> 정확도:{acc_xgb:.4f}, 정밀도:{prec_xgb:.4f}, 재현율:{reca_xgb:.4f}')
print(f'lgb -> 정확도:{acc_lgb:.4f}, 정밀도:{prec_lgb:.4f}, 재현율:{reca_lgb:.4f}')

dtc -> 정확도:0.7468, 정밀도:0.6364, 재현율:0.6481
rfc -> 정확도:0.7532, 정밀도:0.6600, 재현율:0.6111
xgb -> 정확도:0.7403, 정밀도:0.6458, 재현율:0.5741
lgb -> 정확도:0.7143, 정밀도:0.6000, 재현율:0.5556
